In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import os

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [3]:
data_path = 'C:\\Users\\An_asus\\Desktop\\Programming\\JupyterNotebook\\pytorch\\data'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [4]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [5]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

In [8]:
os.getcwd()

'C:\\Users\\An_asus\\Desktop\\Programming\\JupyterNotebook\\pytorch\\data'

In [6]:
import matplotlib.pyplot as plt

In [7]:
model = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=(3, 3), padding=1),
        nn.Tanh(),
        nn.MaxPool2d(2),
        nn.Conv2d(16, 8, kernel_size=(3, 3), padding=1),
        nn.Tanh(),
        nn.MaxPool2d(2),
        nn.Flatten(),
        nn.Linear(8 * 8 * 8, 32),
        nn.Tanh(),
        nn.Linear(32, 2)
)


In [8]:
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list


(18090, [432, 16, 1152, 8, 16384, 32, 64, 2])

In [9]:
img, _ = cifar2[0]

In [10]:
model(img.unsqueeze(0))

C:\Users\An_asus\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


tensor([[-0.0619, -0.1190]], grad_fn=<AddmmBackward>)

In [11]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=(3, 3), padding=1)
        self.act = nn.Tanh()
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=(3, 3), padding=1)
        self.fc1 = nn.Linear(8*8*8, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, t):
        x = self.pool(self.act(self.conv1(t)))
        x = self.pool(self.act(self.conv2(x)))
        x = x.view(-1, 8*8*8)
        x = self.act(self.fc1(x))
        return self.fc2(x)

In [12]:
model = Net()
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list


(18090, [432, 16, 1152, 8, 16384, 32, 64, 2])

In [13]:
import torch.nn.functional as F

In [86]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=(3, 3), padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=(3, 3), padding=1)
        self.fc1 = nn.Linear(8*8*8, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, t):
        x = F.max_pool2d(F.tanh(self.conv1(t)), 2)
        x = F.max_pool2d(F.tanh(self.conv2(x)), 2)
        with torch.no_grad():
            fst_linear_shape = torch.prod(torch.tensor(x.shape[1:]))
        x = x.view(-1, fst_linear_shape)
        x = F.tanh(self.fc1(x))
        return self.fc2(x)

In [106]:
model = Net().to('cuda')

In [40]:
import datetime

In [72]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            
            imgs = imgs.to('cuda')
            labels = labels.to('cuda')
            
            t_p = model(imgs)
            
            loss = loss_fn(t_p, labels)
            
            optimizer.zero_grad()
            
            loss.backward()
            
            optimizer.step()
            
            loss_train += loss.item()
            
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
            datetime.datetime.now(), epoch,
            loss_train / len(train_loader)))

In [95]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size = 64, shuffle=True)

In [107]:
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [97]:
training_loop(n_epochs = 100,
              optimizer = optimizer,
              model = model,
              loss_fn = loss_fn,
              train_loader= train_loader)

2021-09-08 03:53:42.327656 Epoch 1, Training loss 0.561052217415184
2021-09-08 03:53:47.080010 Epoch 10, Training loss 0.34501009553101414
2021-09-08 03:53:52.549418 Epoch 20, Training loss 0.30996516289984344
2021-09-08 03:53:58.326850 Epoch 30, Training loss 0.28421573312419235
2021-09-08 03:54:03.846189 Epoch 40, Training loss 0.25976174785073397
2021-09-08 03:54:09.401607 Epoch 50, Training loss 0.23997293687929774
2021-09-08 03:54:15.328046 Epoch 60, Training loss 0.2207812497949904
2021-09-08 03:54:20.923464 Epoch 70, Training loss 0.2029385419598051
2021-09-08 03:54:26.641892 Epoch 80, Training loss 0.187341618547394
2021-09-08 03:54:32.342875 Epoch 90, Training loss 0.17050105693993295
2021-09-08 03:54:37.954296 Epoch 100, Training loss 0.15981064658540828


In [98]:
train_loader_test = torch.utils.data.DataLoader(cifar2, batch_size = 64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size= 64, shuffle=False)

In [99]:
def validate(model, train_loader_test, val_loader):
    for name, loader in [('train', train_loader_test), ('val', val_loader)]:
        correct = 0
        total = 0
        
        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to('cuda')
                labels = labels.to('cuda')
                
                t_p = model(imgs)
                
                _, predicted = torch.max(t_p, dim=1)
                
                total += labels.shape[0]
                correct += int((predicted == labels).sum())
            
        print("Accuracy {}: {:.2f}".format(name , correct / total))
                

In [100]:
validate(model, train_loader_test, val_loader)

Accuracy train: 0.94
Accuracy val: 0.88


In [50]:
os.getcwd()

'C:\\Users\\An_asus\\Desktop\\Programming\\JupyterNotebook\\pytorch'

In [52]:
os.mkdir('pytorch_models')

In [54]:
os.chdir('pytorch_models/')

In [67]:
saved_model_path = 'C:\\Users\\An_asus\\Desktop\\Programming\\JupyterNotebook\\pytorch\\pytorch_models\\'

In [68]:
torch.save(model.state_dict(), saved_model_path + 'birds_vs_planes.pt')

In [69]:
saved_model_path

'C:\\Users\\An_asus\\Desktop\\Programming\\JupyterNotebook\\pytorch\\pytorch_models\\'

In [71]:
loaded_model = Net()
loaded_model.load_state_dict(torch.load(saved_model_path + 'birds_vs_planes.pt'))

<All keys matched successfully>

In [75]:
sum(p.numel() for p in model.parameters())

18090

In [105]:
def training_loop_l2reg(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs+1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            
            imgs = imgs.to('cuda')
            labels = labels.to('cuda')
            
            t_p = model(imgs)
            
            loss = loss_fn(t_p, labels)
            
            l2_lambda = 0.001
            
            l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
            
            loss = loss + l2_lambda * l2_norm
            
            optimizer.zero_grad()
            
            loss.backward()
            
            optimizer.step()
            
            loss_train += loss.item()
            
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [108]:
training_loop_l2reg(n_epochs = 100,
                      optimizer = optimizer,
                      model = model,
                      loss_fn = loss_fn,
                      train_loader= train_loader)

C:\Users\An_asus\Anaconda3\lib\site-packages\torch\nn\functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


2021-09-08 03:57:40.018954 Epoch 1, Training loss 0.5769738627087538
2021-09-08 03:57:46.423431 Epoch 10, Training loss 0.3598576806912756
2021-09-08 03:57:53.534963 Epoch 20, Training loss 0.32333577685295395
2021-09-08 03:58:01.142527 Epoch 30, Training loss 0.29841613399375017
2021-09-08 03:58:08.569085 Epoch 40, Training loss 0.281171115720348
2021-09-08 03:58:16.388664 Epoch 50, Training loss 0.2649215316506708
2021-09-08 03:58:24.038233 Epoch 60, Training loss 0.2519856596448619
2021-09-08 03:58:31.878818 Epoch 70, Training loss 0.240795052450174
2021-09-08 03:58:39.535390 Epoch 80, Training loss 0.2331613962817344
2021-09-08 03:58:47.421459 Epoch 90, Training loss 0.2234895055176346
2021-09-08 03:58:55.127032 Epoch 100, Training loss 0.21384811724067493


In [109]:
validate(model, train_loader_test, val_loader)

Accuracy train: 0.92
Accuracy val: 0.89


In [116]:
class NetDropout(nn.Module):
    def __init__(self, n_chans=32):
        super(NetDropout, self).__init__()
        self.n_chans = n_chans
        self.conv1 = nn.Conv2d(3, n_chans, kernel_size=(3, 3), padding=1)
        self.conv2 = nn.Conv2d(n_chans, n_chans // 2, kernel_size=(3, 3), padding=1)
        self.convdropout = nn.Dropout2d(p=0.4)
        self.fc1 = nn.Linear(8 * 8 * n_chans // 2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, t):
        x = F.max_pool2d(F.tanh(self.conv1(t)), 2)
        x = self.convdropout(x)
        x = F.max_pool2d(F.tanh(self.conv2(x)), 2)
        x = self.convdropout(x)
        
        x = x.view(-1, 8 * 8 * self.n_chans // 2)
        x = F.tanh(self.fc1(x))
        return self.fc2(x)

In [120]:
model_drop = NetDropout().to('cuda')

optimizer = optim.SGD(model_drop.parameters(), lr=1e-2)

In [118]:
training_loop(n_epochs=100,
              optimizer=optimizer,
              loss_fn=loss_fn,
              model= model_drop,
              train_loader=train_loader)

2021-09-08 04:09:53.686455 Epoch 1, Training loss 0.5670157640602937
2021-09-08 04:10:00.078931 Epoch 10, Training loss 0.3752840326470175
2021-09-08 04:10:06.851631 Epoch 20, Training loss 0.34575967576093736
2021-09-08 04:10:13.818150 Epoch 30, Training loss 0.32157994711854654
2021-09-08 04:10:20.572653 Epoch 40, Training loss 0.3106063318670176
2021-09-08 04:10:27.547172 Epoch 50, Training loss 0.2907200011478108
2021-09-08 04:10:34.243378 Epoch 60, Training loss 0.2771661716281988
2021-09-08 04:10:40.896874 Epoch 70, Training loss 0.2659934281258826
2021-09-08 04:10:47.761151 Epoch 80, Training loss 0.25956444736498935
2021-09-08 04:10:54.590658 Epoch 90, Training loss 0.2458324297598213
2021-09-08 04:11:01.575187 Epoch 100, Training loss 0.23956219785532373


In [119]:
validate(model_drop, train_loader_test, val_loader)

C:\Users\An_asus\Anaconda3\lib\site-packages\torch\nn\functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Accuracy train: 0.91
Accuracy val: 0.87


In [121]:
training_loop_l2reg(n_epochs=100,
              optimizer=optimizer,
              loss_fn=loss_fn,
              model= model_drop,
              train_loader=train_loader)

C:\Users\An_asus\Anaconda3\lib\site-packages\torch\nn\functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


2021-09-08 04:11:36.151785 Epoch 1, Training loss 0.6052124557221771
2021-09-08 04:11:44.042586 Epoch 10, Training loss 0.4036009721695238
2021-09-08 04:11:52.829241 Epoch 20, Training loss 0.3788613119892254
2021-09-08 04:12:01.395879 Epoch 30, Training loss 0.35749595654997857
2021-09-08 04:12:10.049726 Epoch 40, Training loss 0.34335598529903755
2021-09-08 04:12:18.652366 Epoch 50, Training loss 0.3282385178052696
2021-09-08 04:12:27.218673 Epoch 60, Training loss 0.3173913386217348
2021-09-08 04:12:35.801713 Epoch 70, Training loss 0.3067009375922999
2021-09-08 04:12:44.230343 Epoch 80, Training loss 0.30013750768770836
2021-09-08 04:12:52.939273 Epoch 90, Training loss 0.28894850013742024
2021-09-08 04:13:01.534760 Epoch 100, Training loss 0.2805487247788982


In [122]:
validate(model_drop, train_loader_test, val_loader)

Accuracy train: 0.90
Accuracy val: 0.88


In [125]:
class NetBatchNorm(nn.Module):
    
    def __init__(self, n_chans =32):
        super(NetBatchNorm, self).__init__()
        
        self.n_chans = n_chans
        self.conv1 = nn.Conv2d(3, n_chans, kernel_size=(3, 3), padding=1)
        self.bn_conv1 = nn.BatchNorm2d(num_features=n_chans)
        
        self.conv2 = nn.Conv2d(n_chans, n_chans // 2, kernel_size=(3, 3), padding=1)
        self.bn_conv2 = nn.BatchNorm2d(num_features=n_chans // 2)
        self.fc1 = nn.Linear(8*8*n_chans // 2, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, t):
        x = self.bn_conv1(self.conv1(t))
        x = F.max_pool2d(torch.tanh(x), 2)
        
        x = self.bn_conv2(self.conv2(x))
        x = F.max_pool2d(torch.tanh(x), 2)
        
        x = x.view(-1, 8*8*self.n_chans // 2)
        x = torch.tanh(self.fc1(x))
        return self.fc2(x)
        

In [132]:
model_bn = NetBatchNorm().to('cuda')

optimizer = optim.SGD(model_bn.parameters(), lr=1e-2)

In [129]:
training_loop(n_epochs=100,
              optimizer=optimizer,
              loss_fn=loss_fn,
              model= model_bn,
              train_loader=train_loader)

2021-09-08 04:23:02.177380 Epoch 1, Training loss 0.4543456488354191
2021-09-08 04:23:09.241832 Epoch 10, Training loss 0.26876255936303717
2021-09-08 04:23:17.215428 Epoch 20, Training loss 0.21226825019356552
2021-09-08 04:23:25.031009 Epoch 30, Training loss 0.16525772854590873
2021-09-08 04:23:33.043607 Epoch 40, Training loss 0.1305396316489976
2021-09-08 04:23:41.046979 Epoch 50, Training loss 0.10075461850234657
2021-09-08 04:23:49.033572 Epoch 60, Training loss 0.07316335196944938
2021-09-08 04:23:56.957580 Epoch 70, Training loss 0.05464557723207458
2021-09-08 04:24:04.938175 Epoch 80, Training loss 0.036121688617073044
2021-09-08 04:24:12.951649 Epoch 90, Training loss 0.03295332788946522
2021-09-08 04:24:20.924244 Epoch 100, Training loss 0.02031807359426644


In [130]:
validate(model_bn, train_loader_test, val_loader)

Accuracy train: 0.94
Accuracy val: 0.84


In [133]:
training_loop_l2reg(n_epochs=100,
                  optimizer=optimizer,
                  loss_fn=loss_fn,
                  model= model_bn,
                  train_loader=train_loader)

2021-09-08 04:25:14.316469 Epoch 1, Training loss 0.5191393557247842
2021-09-08 04:25:23.910185 Epoch 10, Training loss 0.3422966164768122
2021-09-08 04:25:34.232954 Epoch 20, Training loss 0.283682864562721
2021-09-08 04:25:44.984756 Epoch 30, Training loss 0.24546698580501944
2021-09-08 04:25:54.887493 Epoch 40, Training loss 0.21193236720030476
2021-09-08 04:26:04.878237 Epoch 50, Training loss 0.18048901006484488
2021-09-08 04:26:14.775973 Epoch 60, Training loss 0.16471033775882357
2021-09-08 04:26:24.391568 Epoch 70, Training loss 0.14520732731006708
2021-09-08 04:26:34.137296 Epoch 80, Training loss 0.13520758494639853
2021-09-08 04:26:44.555774 Epoch 90, Training loss 0.12072904597801767
2021-09-08 04:26:54.183395 Epoch 100, Training loss 0.1107212676648881


In [134]:
validate(model_bn, train_loader_test, val_loader)

Accuracy train: 0.99
Accuracy val: 0.89


In [135]:
class NetDepth(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3,
        padding=1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2,
        kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * n_chans1 // 2, 32)
        self.fc2 = nn.Linear(32, 2)
    def forward(self, t):
        x1 = F.max_pool2d(torch.relu(self.conv1(t)), 2)
        x = F.max_pool2d(torch.relu(self.conv2(x1)), 2)
        x = F.max_pool2d(torch.relu(self.conv3(x + x1)), 2)
        
        x = x.view(-1, 4*4* self.n_chans1 // 2)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

In [136]:
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=(3, 3), padding=1)
        self.bn = nn.BatchNorm2d(num_features=n_chans)
        
    def forward(self, t):
        x = torch.relu(self.bn(self.conv(t)))
        return x + t

In [146]:
class NetResDeep(nn.Module):
    def __init__(self, n_chans1 = 32, n_blocks = 10):
        super(NetResDeep, self).__init__()
        
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=(3, 3), padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=n_chans1)
        
        self.resblocks = nn.Sequential( * (n_blocks * [ResBlock(n_chans=n_chans1)]))
        
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 2)
    
    def forward(self, t):
        x = self.bn1(self.conv1(t))
        x = F.max_pool2d(torch.relu(x), 2)
        x = self.resblocks(x)
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 8*8*self.n_chans1)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)


In [147]:
res_model = NetResDeep(n_chans1 = 16, n_blocks=5).to('cuda')

res_optimizer = optim.Adam(res_model.parameters(), lr=3e-3)

training_loop_l2reg(n_epochs=50, 
                    optimizer = res_optimizer, 
                    model = res_model, 
                    loss_fn=loss_fn,
                    train_loader=train_loader)

2021-09-08 04:59:01.476997 Epoch 1, Training loss 0.608213076545934
2021-09-08 04:59:14.496821 Epoch 10, Training loss 0.27595854688222243
2021-09-08 04:59:28.394856 Epoch 20, Training loss 0.2313645642938887
2021-09-08 04:59:42.652776 Epoch 30, Training loss 0.20794062767249005
2021-09-08 04:59:56.827831 Epoch 40, Training loss 0.1962177941848518
2021-09-08 05:00:11.599387 Epoch 50, Training loss 0.18170336871200307


In [148]:
validate(res_model, train_loader_test, val_loader)

Accuracy train: 0.96
Accuracy val: 0.91


In [153]:
torch.save(res_model.state_dict(), saved_model_path + 'ResNet_plane vs bird.pt')